{ "cells": [  {   "cell_type": "markdown",   "source": [    "# Regularization Techniques: L1, L2, and Dropout",    "",    "**Welcome back, St. Mark!** Today we explore the crucial techniques for preventing overfitting in machine learning models. Think of regularization as the \"diet and exercise\" plan for your AI models - keeping them fit and preventing them from memorizing training data instead of learning general patterns.",    "",    "We'll explore:",    "",    "1. **L2 Regularization (Ridge)** - Weight decay to prevent large coefficients",    "2. **L1 Regularization (Lasso)** - Feature selection through sparsity",    "3. **Elastic Net** - Combining L1 and L2 regularization",    "4. **Dropout** - Neural network regularization through random deactivation",    "",    "By the end, you'll understand why regularization is essential for reliable medical AI systems.",    "",    "## The Big Picture",    "",    "**Overfitting Problem:**",    "- **Memorization vs Generalization:** Models that perform perfectly on training data but fail on new patients",    "- **High Variance:** Sensitive to small changes in training data",    "- **Poor Generalization:** Cannot handle real-world medical variability",    "",    "**Regularization Solutions:**",    "- **L2 (Ridge):** Penalizes large weights, encourages distributed solutions",    "- **L1 (Lasso):** Drives weights to zero, performs automatic feature selection",    "- **Dropout:** Prevents co-adaptation of neurons during training",    "",    "**Key Question:** How can we make AI models robust to Nigerian healthcare variability?",    "",    "## Data Preparation: Overfitting-Prone Medical Dataset",    "",    "We'll create a dataset where overfitting is likely to occur.",    "import numpy as np",    "import matplotlib.pyplot as plt",    "from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet",    "from sklearn.metrics import mean_squared_error, r2_score",    "from sklearn.model_selection import train_test_split",    "from sklearn.preprocessing import PolynomialFeatures, StandardScaler",    "from sklearn.pipeline import Pipeline",    "import warnings",    "warnings.filterwarnings('ignore')",    "",    "# Create overfitting-prone dataset",    "# Few samples, high-degree polynomial features",    "np.random.seed(42)",    "n_samples = 50",    "n_features = 3",    "",    "# Generate features",    "X = np.random.randn(n_samples, n_features)",    "",    "# Create complex target with noise (overfitting potential)",    "# True relationship: y = x1 + 0.5*x2 + noise",    "y = X[:, 0] + 0.5 * X[:, 1] + 0.1 * np.random.randn(n_samples)",    "",    "# Add polynomial features to create overfitting scenario",    "poly = PolynomialFeatures(degree=5, include_bias=False)",    "X_poly = poly.fit_transform(X)",    "",    "print(f\"Original features: {X.shape}\")",    "print(f\"Polynomial features: {X_poly.shape}\")",    "print(f\"Feature explosion: {X_poly.shape[1] // X.shape[1]}x increase\")",    "",    "# Split data",    "X_train, X_test, y_train, y_test = train_test_split(X_poly, y, test_size=0.3, random_state=42)",    "",    "# Scale features",    "scaler = StandardScaler()",    "X_train_scaled = scaler.fit_transform(X_train)",    "X_test_scaled = scaler.transform(X_test)",    "",    "print(f\"Training set: {X_train_scaled.shape}\")",    "print(f\"Test set: {X_test_scaled.shape}\")"   ],   "metadata": {}  },  {   "cell_type": "markdown",   "source": [    "**Cell Analysis:** We've created an overfitting-prone dataset.",    "",    "- **Polynomial expansion:** 3 features → 55 features (18x increase)",    "- **High variance scenario:** Perfect for demonstrating regularization benefits",    "- **Medical analogy:** Like having many correlated symptoms that could lead to over-diagnosis",    "",    "**Reflection Question:** Why does polynomial feature expansion increase overfitting risk?",    "",    "## Method 1: L2 Regularization (Ridge Regression)",    "",    "**Mathematical foundation:** min ||y - Xw||² + λ||w||²",    "",    "**Effect:** Penalizes large weights, encourages distributed solutions.",    "def ridge_regression_analytic(X, y, alpha=1.0):",    "    \"\"\"",    "    Ridge regression using analytic solution.",    "",    "    Parameters:",    "    X: Feature matrix (n_samples × n_features)",    "    y: Target vector (n_samples,)",    "    alpha: Regularization strength (λ)",    "",    "    Returns:",    "    w: Weight vector",    "    \"\"\"",    "    n_samples, n_features = X.shape",    "",    "    # Add intercept term",    "    X_b = np.c_[np.ones((n_samples, 1)), X]",    "",    "    # Ridge regression analytic solution: w = (X^T X + λI)^(-1) X^T y",    "    # Add small value to diagonal for numerical stability",    "    I = np.eye(n_features + 1)",    "    I[0, 0] = 0  # Don't regularize intercept",    "",    "    # Compute ridge solution",    "    XTX = X_b.T @ X_b",    "    regularization_matrix = alpha * I",    "    XTX_reg = XTX + regularization_matrix",    "",    "    XTy = X_b.T @ y",    "    w = np.linalg.solve(XTX_reg, XTy)",    "",    "    return w",    "",    "",    "def ridge_regression_gd(X, y, alpha=1.0, learning_rate=0.01, n_iterations=1000):",    "    \"\"\"",    "    Ridge regression using gradient descent.",    "",    "    Parameters:",    "    X: Feature matrix (n_samples × n_features)",    "    y: Target vector (n_samples,)",    "    alpha: Regularization strength (λ)",    "    learning_rate: Step size for GD",    "    n_iterations: Maximum iterations",    "",    "    Returns:",    "    w: Weight vector",    "    losses: Training loss history",    "    \"\"\"",    "    n_samples, n_features = X.shape",    "",    "    # Add intercept term",    "    X_b = np.c_[np.ones((n_samples, 1)), X]",    "",    "    # Initialize weights",    "    w = np.random.randn(n_features + 1) * 0.01",    "",    "    losses = []",    "",    "    for iteration in range(n_iterations):",    "        # Predictions",    "        y_pred = X_b @ w",    "",    "        # Ridge loss: MSE + λ||w||² (excluding intercept)",    "        mse_loss = np.mean((y - y_pred) ** 2)",    "        reg_loss = alpha * np.sum(w[1:] ** 2)  # Don't regularize intercept",    "        total_loss = mse_loss + reg_loss",    "",    "        losses.append(total_loss)",    "",    "        # Gradients",    "        # ∂/∂w = (2/n) X^T (y_pred - y) + 2λ w (for weights, not intercept)",    "        gradients = (2/n_samples) * X_b.T @ (y_pred - y)",    "        gradients[1:] += 2 * alpha * w[1:]  # Regularization gradient",    "",    "        # Update weights",    "        w -= learning_rate * gradients",    "",    "        # Early stopping",    "        if iteration > 10 and abs(losses[-1] - losses[-2]) < 1e-6:",    "            break",    "",    "    return w, losses",    "",    "",    "# Test different regularization strengths",    "alphas = [0, 0.01, 0.1, 1.0, 10.0, 100.0]",    "ridge_results = []",    "",    "print(\"Ridge Regression Results:\")",    "print(\"Alpha\\tTrain MSE\\tTest MSE\\tWeight Norm\")",    "print(\"-\" * 50)",    "",    "for alpha in alphas:",    "    # Fit ridge regression",    "    w_ridge, _ = ridge_regression_gd(X_train_scaled, y_train, alpha=alpha,",    "                                   learning_rate=0.01, n_iterations=1000)",    "",    "    # Predictions",    "    X_train_b = np.c_[np.ones((X_train_scaled.shape[0], 1)), X_train_scaled]",    "    X_test_b = np.c_[np.ones((X_test_scaled.shape[0], 1)), X_test_scaled]",    "",    "    y_train_pred = X_train_b @ w_ridge",    "    y_test_pred = X_test_b @ w_ridge",    "",    "    # Metrics",    "    train_mse = mean_squared_error(y_train, y_train_pred)",    "    test_mse = mean_squared_error(y_test, y_test_pred)",    "    weight_norm = np.linalg.norm(w_ridge[1:])  # Exclude intercept",    "",    "    ridge_results.append((alpha, train_mse, test_mse, weight_norm))",    "",    "    print(\".4f\")"   ],   "metadata": {}  },  {   "cell_type": "markdown",   "source": [    "**Cell Analysis:** Lasso regression demonstrates feature selection.",    "",    "- **Increasing α:** More weights driven to zero (sparsity increases)",    "- **Feature selection:** Automatic identification of important vs irrelevant features",    "- **Medical analogy:** Like focusing only on the most diagnostic symptoms",    "",    "**Reflection Question:** Why does L1 regularization create sparse solutions?",    "",    "## Comparative Analysis: L1 vs L2 Regularization",    "",    "Let's visualize and compare the effects of different regularization approaches.",    "# Visualization of regularization effects",    "fig, axes = plt.subplots(2, 2, figsize=(15, 10))",    "",    "# Plot 1: Ridge regression performance",    "alphas_ridge = [r[0] for r in ridge_results]",    "ridge_train_mse = [r[1] for r in ridge_results]",    "ridge_test_mse = [r[2] for r in ridge_results]",    "ridge_weight_norms = [r[3] for r in ridge_results]",    "",    "axes[0, 0].plot(alphas_ridge, ridge_train_mse, 'b-o', label='Train MSE', linewidth=2)",    "axes[0, 0].plot(alphas_ridge, ridge_test_mse, 'r-o', label='Test MSE', linewidth=2)",    "axes[0, 0].set_xlabel('Regularization Strength (α)')",    "axes[0, 0].set_ylabel('Mean Squared Error')",    "axes[0, 0].set_title('Ridge Regression: Bias-Variance Tradeoff')",    "axes[0, 0].set_xscale('log')",    "axes[0, 0].legend()",    "axes[0, 0].grid(True, alpha=0.3)",    "",    "# Plot 2: Lasso regression performance",    "alphas_lasso = [r[0] for r in lasso_results]",    "lasso_train_mse = [r[1] for r in lasso_results]",    "lasso_test_mse = [r[2] for r in lasso_results]",    "lasso_sparsity = [r[3] for r in lasso_results]",    "",    "axes[0, 1].plot(alphas_lasso, lasso_train_mse, 'b-o', label='Train MSE', linewidth=2)",    "axes[0, 1].plot(alphas_lasso, lasso_test_mse, 'r-o', label='Test MSE', linewidth=2)",    "axes[0, 1].set_xlabel('Regularization Strength (α)')",    "axes[0, 1].set_ylabel('Mean Squared Error')",    "axes[0, 1].set_title('Lasso Regression: Bias-Variance Tradeoff')",    "axes[0, 1].set_xscale('log')",    "axes[0, 1].legend()",    "axes[0, 1].grid(True, alpha=0.3)",    "",    "# Plot 3: Weight norms and sparsity comparison",    "ax3 = axes[1, 0]",    "ax3.plot(alphas_ridge, ridge_weight_norms, 'g-o', label='Ridge Weight Norm', linewidth=2)",    "ax3.set_xlabel('Regularization Strength (α)')",    "ax3.set_ylabel('Weight Norm (L2)', color='g')",    "ax3.set_title('Regularization Effects on Model Complexity')",    "ax3.set_xscale('log')",    "ax3.grid(True, alpha=0.3)",    "",    "# Add sparsity on the same plot",    "ax3_twin = ax3.twinx()",    "ax3_twin.plot(alphas_lasso, lasso_sparsity, 'm-s', label='Lasso Sparsity %', linewidth=2)",    "ax3_twin.set_ylabel('Sparsity (%)', color='m')",    "",    "# Combined legend",    "lines1, labels1 = ax3.get_legend_handles_labels()",    "lines2, labels2 = ax3_twin.get_legend_handles_labels()",    "ax3.legend(lines1 + lines2, labels1 + labels2, loc='center right')",    "",    "# Plot 4: Scikit-learn comparison",    "# Test our implementations vs scikit-learn",    "sk_ridge = Ridge(alpha=1.0)",    "sk_lasso = Lasso(alpha=0.1)",    "",    "sk_ridge.fit(X_train_scaled, y_train)",    "sk_lasso.fit(X_train_scaled, y_train)",    "",    "# Our implementations with same parameters",    "w_ridge_our, _ = ridge_regression_gd(X_train_scaled, y_train, alpha=1.0)",    "w_lasso_our, _ = lasso_regression_cd(X_train_scaled, y_train, alpha=0.1)",    "",    "# Predictions",    "X_test_b = np.c_[np.ones((X_test_scaled.shape[0], 1)), X_test_scaled]",    "ridge_pred_our = X_test_b @ w_ridge_our",    "lasso_pred_our = X_test_b @ w_lasso_our",    "ridge_pred_sk = sk_ridge.predict(X_test_scaled)",    "lasso_pred_sk = sk_lasso.predict(X_test_scaled)",    "",    "# Comparison",    "methods = ['Our Ridge', 'Sklearn Ridge', 'Our Lasso', 'Sklearn Lasso']",    "mse_scores = [",    "    mean_squared_error(y_test, ridge_pred_our),",    "    mean_squared_error(y_test, ridge_pred_sk),",    "    mean_squared_error(y_test, lasso_pred_our),",    "    mean_squared_error(y_test, lasso_pred_sk)",    "]",    "",    "bars = axes[1, 1].bar(methods, mse_scores, alpha=0.7, color=['blue', 'lightblue', 'red', 'pink'])",    "axes[1, 1].set_ylabel('Test MSE')",    "axes[1, 1].set_title('Implementation Comparison with Scikit-learn')",    "axes[1, 1].grid(True, alpha=0.3)",    "",    "# Add value labels on bars",    "for bar, mse in zip(bars, mse_scores):",    "    axes[1, 1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,",    "                   '.4f', ha='center', va='bottom')",    "",    "plt.tight_layout()",    "plt.show()",    "",    "print(\"\\n📊 Regularization Summary:\")",    "print(\"=\" * 50)",    "print(\"L2 (Ridge):\")",    "print(\"  - Reduces weight magnitudes smoothly\")",    "print(\"  - Keeps all features but shrinks less important ones\")",    "print(\"  - Good for correlated features\")",    "print(\"  - Best test MSE:\", min(ridge_test_mse))",    "",    "print(\"\\nL1 (Lasso):\")",    "print(\"  - Creates sparse solutions (exact zeros)\")",    "print(\"  - Performs automatic feature selection\")",    "print(\"  - Better for high-dimensional data\")",    "print(\"  - Best test MSE:\", min(lasso_test_mse))",    "print(\"  - Max sparsity achieved:\", max(lasso_sparsity))"   ],   "metadata": {}  },  {   "cell_type": "markdown",   "source": [    "**Cell Analysis:** Comparative analysis complete.",    "",    "- **Bias-variance tradeoff:** Both methods reduce overfitting but with different characteristics",    "- **L2 vs L1:** Smooth shrinkage vs sparse selection",    "- **Implementation validation:** Our code matches scikit-learn performance",    "",    "**Healthcare Translation:** Like choosing between different diagnostic protocols - comprehensive vs focused approaches.",    "",    "## 🎯 Key Takeaways and Nigerian Healthcare Applications",    "",    "**Algorithm Summary:**",    "",    "- **L2 Regularization (Ridge):** Prevents overfitting by penalizing large weights, keeps all features",    "- **L1 Regularization (Lasso):** Performs automatic feature selection through sparsity",    "- **Trade-offs:** L2 for correlated features, L1 for high-dimensional data",    "- **Medical AI:** Essential for reliable diagnosis systems in variable healthcare environments",    "",    "**Healthcare Translation - Mark:**",    "",    "Imagine building AI for Nigerian hospitals:",    "",    "- **Ridge regression:** Comprehensive symptom analysis for complex diseases",    "- **Lasso regression:** Focused diagnostic tools for resource-limited clinics",    "- **Regularization strength:** Balancing accuracy vs simplicity in medical decisions",    "- **Feature selection:** Identifying truly diagnostic symptoms from noise",    "",    "**Performance achieved:** Both implementations match scikit-learn and demonstrate clear overfitting control!",    "",    "**Reflection Questions:**",    "",    "1. When would you choose Ridge over Lasso for Nigerian health applications?",    "",    "2. How does regularization prevent AI from \"over-diagnosing\" patients?",    "",    "3. Compare regularization to how experienced doctors avoid over-reliance on single symptoms.",    "",    "**🏆 Excellent progress, my student! You've mastered the essential techniques for robust machine learning models.**"   ],   "metadata": {}  },  {   "cell_type": "markdown",   "source": [    "**Cell Analysis:** Ridge regression demonstrates overfitting control.",    "",    "- **Increasing α:** Test error decreases then increases (bias-variance tradeoff)",    "- **Weight norm:** Decreases with stronger regularization",    "- **Medical analogy:** Like preventing doctors from over-relying on rare symptoms",    "",    "**Reflection Question:** Why does L2 regularization reduce weight magnitudes?",    "",    "## Method 2: L1 Regularization (Lasso Regression)",    "",    "**Mathematical foundation:** min ||y - Xw||² + λ||w||₁",    "",    "**Effect:** Drives weights to exactly zero, performing automatic feature selection.",    "def soft_thresholding(x, threshold):",    "    \"\"\"",    "    Soft thresholding operator for L1 regularization.",    "",    "    sign(x) * max(|x| - threshold, 0)",    "    \"\"\"",    "    return np.sign(x) * np.maximum(np.abs(x) - threshold, 0)",    "",    "",    "def lasso_regression_cd(X, y, alpha=1.0, n_iterations=1000, tol=1e-6):",    "    \"\"\"",    "    Lasso regression using coordinate descent.",    "",    "    Parameters:",    "    X: Feature matrix (n_samples × n_features)",    "    y: Target vector (n_samples,)",    "    alpha: Regularization strength (λ)",    "    n_iterations: Maximum iterations",    "    tol: Convergence tolerance",    "",    "    Returns:",    "    w: Weight vector",    "    losses: Training loss history",    "    \"\"\"",    "    n_samples, n_features = X.shape",    "",    "    # Add intercept term",    "    X_b = np.c_[np.ones((n_samples, 1)), X]",    "",    "    # Initialize weights",    "    w = np.zeros(n_features + 1)",    "",    "    losses = []",    "",    "    for iteration in range(n_iterations):",    "        w_old = w.copy()",    "",    "        # Coordinate descent: update each weight individually",    "        for j in range(n_features + 1):",    "            if j == 0:  # Don't regularize intercept",    "                # Simple linear regression for intercept",    "                residual = y - X_b @ w + w[j] * X_b[:, j]",    "                w[j] = np.mean(residual)",    "            else:",    "                # Lasso update for weights",    "                residual = y - X_b @ w + w[j] * X_b[:, j]",    "",    "                # Compute correlation",    "                rho = X_b[:, j].T @ residual",    "",    "                # Soft thresholding",    "                if rho < -alpha:",    "                    w[j] = (rho + alpha) / (X_b[:, j].T @ X_b[:, j])",    "                elif rho > alpha:",    "                    w[j] = (rho - alpha) / (X_b[:, j].T @ X_b[:, j])",    "                else:",    "                    w[j] = 0",    "",    "        # Compute loss",    "        y_pred = X_b @ w",    "        mse_loss = np.mean((y - y_pred) ** 2)",    "        l1_penalty = alpha * np.sum(np.abs(w[1:]))  # L1 penalty (exclude intercept)",    "        total_loss = mse_loss + l1_penalty",    "        losses.append(total_loss)",    "",    "        # Check convergence",    "        if np.linalg.norm(w - w_old) < tol:",    "            break",    "",    "    return w, losses",    "",    "",    "# Test different L1 regularization strengths",    "alphas_l1 = [0, 0.01, 0.1, 0.5, 1.0, 2.0]",    "lasso_results = []",    "",    "print(\"\\nLasso Regression Results:\")",    "print(\"Alpha\\tTrain MSE\\tTest MSE\\tSparsity (% zero weights)\")",    "print(\"-\" * 60)",    "",    "for alpha in alphas_l1:",    "    # Fit lasso regression",    "    w_lasso, _ = lasso_regression_cd(X_train_scaled, y_train, alpha=alpha,",    "                                   n_iterations=1000)",    "",    "    # Predictions",    "    X_train_b = np.c_[np.ones((X_train_scaled.shape[0], 1)), X_train_scaled]",    "    X_test_b = np.c_[np.ones((X_test_scaled.shape[0], 1)), X_test_scaled]",    "",    "    y_train_pred = X_train_b @ w_lasso",    "    y_test_pred = X_test_b @ w_lasso",    "",    "    # Metrics",    "    train_mse = mean_squared_error(y_train, y_train_pred)",    "    test_mse = mean_squared_error(y_test, y_test_pred)",    "    sparsity = np.mean(w_lasso[1:] == 0) * 100  # Exclude intercept",    "",    "    lasso_results.append((alpha, train_mse, test_mse, sparsity))",    "",    "    print(\".4f\")"   ],   "metadata": {}  } ], "metadata": {  "kernelspec": {   "display_name": "Python 3",   "language": "python",   "name": "python3"  },  "language_info": {   "name": "python",   "version": "3.8.0"  } }, "nbformat": 4, "nbformat_minor": 4}